In [23]:
pip install scikit-learn

In [1]:
import splitfolders

In [5]:
input_folder=r"C:\Users\ashis\OneDrive\Desktop\dataset2\InputDataset"
output=r"C:\Users\ashis\OneDrive\Desktop\dataset2\processed_Output"
splitfolders.ratio(input_folder,output,seed=42,ratio=(0.7,0.15,0.15))

Copying files: 11546 files [06:09, 31.24 files/s]


In [6]:
help(splitfolders.ratio)

Help on function ratio in module splitfolders.split:

ratio(input, output='output', seed=1337, ratio=(0.8, 0.1, 0.1), group_prefix=None, move=False)



In [3]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
import numpy as np
img_height, img_width = (224, 224)
def preprocess_inputt(img_path):
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img = image.img_to_array(img)
    img = preprocess_input(img)
    
    # Apply bilinear interpolation
    img = cv2.resize(img, (img_height, img_width), interpolation=cv2.INTER_LINEAR)
    
    # Apply bilateral filter
    img = cv2.bilateralFilter(img, 9, 75, 75)
    
    # Convert image to grayscale
    img_gray = rgb2gray(img)
    
    # Apply fuzzy c-means segmentation
    segments_fcm = fuzzy(img_gray, 2, 2, error=0.01, maxiter=1000)[0]
    
    # Threshold the fuzzy c-means segments to obtain a binary mask
    threshold = 0.5
    mask = segments_fcm > threshold
    
    # Apply mask to original image
    img = img * np.expand_dims(mask, axis=-1)
    
    return img

batch_size = 70
train_data_dir = r"C:\Users\ashis\OneDrive\Desktop\dataset2\processed_Output\train"
valid_data_dir = r"C:\Users\ashis\OneDrive\Desktop\dataset2\processed_Output\val"
test_data_dir = r"C:\Users\ashis\OneDrive\Desktop\dataset2\processed_Output\test"

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.4)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

valid_generator = train_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

test_generator = train_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=1,
    class_mode='categorical',
    subset='validation')

x, y = test_generator.next()
x.shape




Found 4850 images belonging to 2 classes.
Found 1154 images belonging to 2 classes.
Found 923 images belonging to 2 classes.


(1, 224, 224, 3)

In [ ]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
import numpy as np
# Load the pre-trained ResNet-50 model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add your own fully connected layers on top of the base model
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
output = Dense(train_generator.num_classes, activation='softmax')(x)  # assuming binary classification (mask vs no-mask)

# Create the fine-tuned model
model = Model(inputs=base_model.input, outputs=output)

# Freeze the base model layers so they are not updated during training
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with an appropriate optimizer, loss function, and evaluation metric
optimizer = Adam(lr=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model using your augmented dataset
history=model.fit(train_generator, epochs=12, batch_size=70, validation_data=valid_generator)

# After fine-tuning, you can unfreeze some of the base model layers if needed, and continue training
# with a lower learning rate to fine-tune further
for layer in base_model.layers[-6:]:
    layer.trainable = True

optimizer = Adam(lr=0.00001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
history2= model.fit(train_generator, epochs=5, batch_size=70, validation_data=valid_generator)

# Evaluate the model on the test data

# Plot the training and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

In [4]:
print(train_generator.num_classes)

2


In [5]:
train_generator.class_indices

{'Mask': 0, 'No mask': 1}

In [8]:
model.save(r"C:\Users\ashis\OneDrive\Desktop\dataset2\Saved models\ResNet50_faceMask.h5")

In [9]:
test_loss,test_acc=model.evaluate(test_generator,verbose=2)
print('\n test accuracy:',test_acc)

923/923 - 136s - loss: 0.1192 - accuracy: 0.9957 - 136s/epoch - 148ms/step

 test accuracy: 0.9956663250923157


In [6]:
import cv2
import numpy as np
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

model = load_model(r"C:\Users\ashis\OneDrive\Desktop\dataset2\Saved models\ResNet50_faceMask.h5")
cap = cv2.VideoCapture(0)  # use 0 for default camera, or specify a camera index if you have multiple cameras

# Define the class labels
class_labels = {'Mask': 0, 'No mask': 1}
labels = dict((v, k) for k, v in class_labels.items())

while True:
    # Capture frame from video stream
    ret, frame = cap.read()

    if ret:
        # Resize frame to match the input size of the model
        frame = cv2.resize(frame, (224, 224))

        # Preprocess the frame for inference
        x = image.img_to_array(frame)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        # Make predictions
        preds = model.predict(x)
        pred_classes = np.argmax(preds, axis=1)

        # Map predicted classes to their corresponding class labels
        pred_class_label = labels[pred_classes[0]]
        # Get the percentage of certainty (confidence score) for the prediction
        confidence = preds[0][pred_classes[0]] * 100

        # Change color to red if predicted class is "No Mask"
        if pred_class_label == "No mask":
            color = (0, 0, 255)  # Red color
        else:
            color = (0, 255, 0)  # Green color

        # Draw the predicted class label and confidence score on the frame
        text = f"{pred_class_label}: {confidence:.2f}%"
        cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        # Display the frame with the predicted class label and confidence score
        cv2.imshow('Face Mask Detection', frame)

    # Exit the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 311ms/step
